<a target="_blank" href="https://colab.research.google.com/github/ArtificialIntelligenceToolkit/aitk/blob/master/notebooks/DemoRobots.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install aitk --upgrade --quiet

ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: 'WHEEL'
Check the permissions.

Note: you may need to restart the kernel to use updated packages.


## Demo of aitk.robots features

Let's create several robots with different capabilities and see how they interact in a small world.

In [2]:
from aitk.robots import World, Scribbler, RangeSensor, LightSensor, SmellSensor, Camera

### A world with walls, a bulb, and food

There is a food source located near the origin of the world behind a small angled blue wall.  There is a light source near the center of the world tucked in the corner of the cyan and orange walls.

In [3]:
world = World(width=200, height=150, scale=5.0)
world.add_wall("cyan", 80, 50, 90, 150)
world.add_wall("orange", 90, 50, 110, 60)
world.add_wall("blue", 0, 35, 25, 30, box=False)
world.add_bulb("yellow", 100, 70, 0, 75.0)
world.add_food(15, 10, 50)

Random seed set to: 1045659


### Red robot uses IR sensors

The red robot uses IR sensors to sense and avoid obstacles. It can determine which way to turn based on whether the obstacle is closer to its front left or front right.

In [4]:
robot1 = Scribbler(x=150, y=100, a=35, color="red", name="red")
robot1.add_device(RangeSensor(position=(6,-6),width=57.3,max=20,name="left-ir"))
robot1.add_device(RangeSensor(position=(6,6),width=57.3,max=20,name="right-ir"))
world.add_robot(robot1)

### Pink robot has smell sensors and a camera

The pink robot uses smell sensors to find food. It also has a camera which can see the features of the world. It can turn towards food sensed to either the left or right.

In [5]:
robot2 = Scribbler(x=40, y=130, a=75, color="pink", name="pink")
robot2.state["timer"] = 0 #use state variable to repeatedly reverse
robot2.add_device(Camera())
robot2.add_device(SmellSensor(position=(6,-6),name="left-smell"))
robot2.add_device(SmellSensor(position=(6,6),name="right-smell"))
world.add_robot(robot2)

### Yellow robot uses light sensors

The yellow robot uses light sensors to find a light source. It can turn towards light sensed to either the left or right.

In [6]:
robot3 = Scribbler(x=60, y=30, a=0, color="yellow", name="yellow")
robot3.add_device(LightSensor(position=(6,-6), name="left-light"))
robot3.add_device(LightSensor(position=(6,6), name="right-light"))
world.add_robot(robot3)

In [7]:
world.watch(width="700px")

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x0…

In [8]:
robot2['camera'].watch()

HTML(value='<style>img.pixelated {image-rendering: pixelated;}</style>')

HTML(value='<style>img.pixelated {image-rendering: pixelated;}</style>')

HTML(value='<style>img.pixelated {image-rendering: pixelated;}</style>')

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00@\x00\x00\x00 \x08\x06\x00\x00\x00\xa2\x9d~\x84\x…

In [9]:
from random import random

### Red robot's controller wanders

The red robot wanders around the world avoiding obstacles it encounters. It always moves forward, but chooses random rotation amounts when its front range sensors are clear.

In [10]:
def wander_ir(robot):
    max_val = robot["left-ir"].get_max()
    if robot["left-ir"].get_distance() < max_val:
        robot.move(0.1, -0.3)
    elif robot["right-ir"].get_distance() < max_val:
        robot.move(0.1, 0.3)
    else:
        robot.move(1, random()-0.5)

### Pink robot's controller wanders

The pink robot wanders around the world searching for food and avoiding obstacles by reversing when stalled.  It uses a state variable called *timer* to ensure that even in the presence of an odor it always takes at least five reverse steps when it encounters an obstacle. If it gets close enough to food, it tries to eat it, and if successful ends the run by returning True. Occasionally it chooses a new random wandering motion.  If none of its test cases are triggered, it repeats the previous motion. 

In [11]:
def search_food(robot):
    left_smell = robot["left-smell"].get_reading()
    right_smell = robot["right-smell"].get_reading()
    diff = left_smell - right_smell
    if left_smell+right_smell>1.95 and robot.eat():
        robot.move(0,0)
        robot.speak("ate food!")
        return True
    if robot.state["timer"] > 5:
        robot.state["timer"] = 0
    if robot.stalled:
        robot.state["timer"] = 1
        robot.reverse()
    elif robot.state["timer"] > 0:
        robot.state["timer"] += 1
    elif diff > 0.03:
        robot.move(1, 0.3)
    elif diff < -0.03:
        robot.move(1, -0.3)
    elif world.time%2 == 0:
        robot.move(1.0, random()*0.5-0.25)
    else:
        pass #continue doing previous action

### Yellow robot's controller seeks light

The yellow robot's goal is to find and approach a light source.  When no light is sensed, the robot wanders. When it senses more light to one side it turns towards the light. If the robot is stalled, it reverses its motion. It occasionally generates a new random wandering motion. When none of the tests are triggered, the robot will continue doing the previous movement. When it's light readings exceed a threshold it stops and reports that it has found the light. By returning True it ends the run. 

In [12]:
def search_light(robot):
    left_light = robot["left-light"].get_brightness()
    right_light = robot["right-light"].get_brightness()
    diff = left_light - right_light
    if left_light + right_light > 1.9:
        robot.move(0,0)
        robot.speak("found light!")
        return True
    if robot.stalled:
        robot.reverse()
    elif diff > 0.01:
        robot.move(1,0.5)
    elif diff < -0.01:
        robot.move(1,-0.5)   
    elif world.time%2 == 0:
        robot.move(1.0, random()*0.5-0.25)
    else:
        pass #continue doing previous action

In [13]:
world.reset()
robot2.state["timer"] = 0 #reset the state variable
world.set_seed(random())
world.seconds(40,[wander_ir,search_food,search_light],real_time=True)

Using random seed: 1045659
Using random seed: 0.8310109303846899


  0%|          | 0/400 [00:00<?, ?it/s]

Simulation stopped at: 00:00:06.50; speed 0.97 x real time
